In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

test_predictions = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
# Principal component analysis (PCA) for cell viability data only
pca_cv = PCA()
pca_cv.fit(train_features.iloc[:,776:876])
labels_cv = ['PC' + str(x) for x in range (1, 101)]

In [ ]:
# Preprocess training and testing features datasets
def preprocess(df):
    df = df.copy()
    
    # PCA - 97 cell viability principal components to replace 100 cell viability variables
    pca_cv_data = pca_cv.transform(df.iloc[:,776:876])
    df = pd.merge(df.iloc[:,:776], pd.DataFrame(pca_cv_data, columns = labels_cv).iloc[:,:97], how = 'inner', left_index = True, right_index = True)

    # Assign numeric labels to categorical values
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 1, 72: 2}) 
    
    # Delete ID column 
    del df['sig_id']
    
    return df

train_features = preprocess(train_features)
test_features = preprocess(test_features)

In [ ]:
# Preprocess training targets datasets
del train_targets['sig_id']

In [ ]:

from sklearn.model_selection import train_test_split
lin_reg = LinearRegression()
lin_reg.fit(train_features , train_targets)

pred = lin_reg.predict(test_features)
test_predictions.iloc[ :, 1:] = pred

test_predictions.to_csv('submission.csv', index = False)


